In [6]:
%pip install langchain langchain-core langgraph

Note: you may need to restart the kernel to use updated packages.


In [59]:
#from define_state_and_llm import State, llm, llm_with_tools
from langchain_core.prompts import PromptTemplate
import json
from langgraph.prebuilt import ToolNode
from typing import TypedDict, List
from langchain_openai import ChatOpenAI
import os
from dotenv import load_dotenv
import requests
from langchain_core.tools import tool
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode
from tavily import TavilyClient

In [10]:
load_dotenv()

class State(TypedDict):
    interest_rate: float | str
    treasury_yield: float | str
    market_rate: float
    num_tool_calls: int
    path: List[str]
    recommendation: str
    #credit_score: int
    #closing_costs: int
    #current_payment: int

llm = ChatOpenAI(model="gpt-4o-mini",
                 api_key=os.getenv("OPENAI_API_KEY"))

In [11]:
def get_treasury_10yr_yield() -> float:
    """"
    Gets the 10 year treasury yield value.
    """

    url = "https://quote.cnbc.com/quote-html-webservice/quote.htm"

    headers = {
    "User-Agent": (
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/118.0.5993.89 Safari/537.36"
    )
    }

    params = {
        "noform": "1",
        "partnerId": "2",
        "fund": "1",
        "exthrs": "0",
        "output": "json",
        "symbols": "US10Y"
    }

    resp = requests.get(url, params=params, headers=headers, timeout=8)
    resp.raise_for_status()
    data = resp.json()

    try:
        quotes = data["QuickQuoteResult"]["QuickQuote"]
        value = quotes[0]["last"]
        str_value = str(value).strip().rstrip("%")
        return float(str_value)
    except Exception as e:
        raise ValueError(f"Unexpected CNBC payload shape or symbol missing: {e}") from e

In [14]:
@tool
def get_treasury_10yr_yield_for_agent() -> float | str:
    """Gets the 10 year treasury yield value."""
    return get_treasury_10yr_yield()

In [31]:
llm_with_tools = llm.bind_tools([get_treasury_10yr_yield_for_agent])

In [43]:
def treasury_yield_agent(state: State) -> dict:
    """Agent gets the latest news articles and summarizes how mortgage rates are at the moment.
    It will also review the 10 year treasury yield rate and see if it's good."""
    prompt = PromptTemplate(template="""You are an expert on treasury yields. You should review the 
                            current 10 Year Treasury Yield rate by calling the `get_treasury_10yr_yield_for_agent` 
                            tool to get the current 10-year Treasury yield value.
                            """)
    resp = llm_with_tools.invoke(prompt.format())
    print(f"\n\n\n*******RESP: {resp}*******\n\n\n")

    tool_node = ToolNode([get_treasury_10yr_yield_for_agent])
    tool_result = tool_node.invoke({"messages": [resp]})
    value = float(tool_result["messages"][0].content)

    print(f"\n\n\n*******tool_result: {tool_result}*******\n\n\n")

    print(f"\n\n\n*******tool_result: {value}*******\n\n\n")

    return state

In [ ]:
    #  resp = llm_with_tools.invoke(prompt.format())
    
    # # Check if LLM wants to call tools
    # if resp.tool_calls:
    #     tool_result = tool_node.invoke({"messages": [resp]})
    #     # Send tool results back to LLM for final response
    #     final_resp = llm_with_tools.invoke([resp] + tool_result["messages"])
    #     content = final_resp.content
    # else:
    #     content = resp.content
    
    # # Parse the JSON response
    # import json
    # parsed = json.loads(content)
    
    # state["treasury_yield"] = parsed["treasury_yield"

In [44]:
workflow = StateGraph(State)
workflow.add_node("treasury_yield_agent_node", treasury_yield_agent)

workflow.set_entry_point("treasury_yield_agent_node")
workflow.add_edge('treasury_yield_agent_node', END)

app=workflow.compile()

In [45]:
initial_state = {
    "interest_rate": 9.0,
    "treasury_yield": 0.0,
    "market_rate": 0.0,
    "recommendation": "",
    "num_tool_calls": 0,
    "path": [""]
}

result = app.invoke(initial_state)




*******RESP: content='' additional_kwargs={'tool_calls': [{'id': 'call_kECpLSIzxJmijdCIR5iXkAA4', 'function': {'arguments': '{}', 'name': 'get_treasury_10yr_yield_for_agent'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 101, 'total_tokens': 120, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'finish_reason': 'tool_calls', 'logprobs': None} id='run--739925ba-0462-4b09-a8ed-458c3d9600bc-0' tool_calls=[{'name': 'get_treasury_10yr_yield_for_agent', 'args': {}, 'id': 'call_kECpLSIzxJmijdCIR5iXkAA4', 'type': 'tool_call'}] usage_metadata={'input_tokens': 101, 'output_tokens': 19, 'total_tokens': 120, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio'

In [79]:
def get_rates_search_tool() -> str:
    """Get the average mortgage interest rate."""

    tavily_client = TavilyClient(api_key=os.getenv('TAVILY_API_KEY'))

    response = tavily_client.search(
        query="""What is the current average mortgage interest rate people in the United States are receiving?" \
        "Provide the answer as a number with 2 decimal places. E.g., 6.55.
        ONLY provide the number without any additional text. ONLY return a single numerical value.""",
        topic="finance",
        search_depth="basic",
        max_results=3,
        time_range="week",
        include_answer=True #Include an LLM-generated answer to the provided query. 
    )

    answer = response["answer"]
    return answer

In [80]:
@tool
def get_rates_search_tool_for_agent() -> str | float:
    """Get the average mortgage interest rate."""
    return get_rates_search_tool()

In [81]:
llm_with_tools = llm.bind_tools([get_treasury_10yr_yield_for_agent, get_rates_search_tool_for_agent])

In [ ]:
def market_expert_agent(state: State) -> dict:
    prompt = PromptTemplate(template="""
                            You are a mortgage market expert. You should summarize some recent articles to get 
                            an average mortgage interest rate people are seeing right now by 
                            calling the `get_rates_search_tool_for_agent`.
                            """)
    prompt_str = prompt.format()
    resp = llm_with_tools.invoke(prompt_str)

    # Check if LLM wants to call tools
    if resp.tool_calls:
        # Execute the tool calls
        tool_node = ToolNode([get_rates_search_tool_for_agent])
        tool_result = tool_node.invoke({"messages": [resp]})
        message = tool_result["messages"][0].content
        print(message)

        # Extract the single value from the text
        follow_on_prompt = f"""Extract the average mortgage interest rate value from this body of text: {message}
                              You must ONLY return the numerical value up to two decimal places.
                              Example answer: 5.32"""
        updated_resp = llm.invoke(follow_on_prompt)
        print(f"UPDATED RESPONSE: {updated_resp.content}")
        return float(updated_resp.content)

    else:
        print("NONE")

    state["num_tool_calls"] += 1
    state["path"].append("market_expert_agent")
    return state

In [92]:
workflow = StateGraph(State)
workflow.add_node("market_expert_node", market_expert_agent)

workflow.set_entry_point("market_expert_node")
workflow.add_edge('market_expert_node', END)

app=workflow.compile()

In [93]:
initial_state = {
    "interest_rate": 9.0,
    "treasury_yield": 0.0,
    "market_rate": 0.0,
    "recommendation": "",
    "num_tool_calls": 0,
    "path": [""]
}

result = app.invoke(initial_state)

As of the current date, the average mortgage interest rate people in the United States are receiving is 6.55 percent. This figure is derived from the analysis provided by Investopedia, which reports the average 30-year fixed rate at 6.50 percent, compared with 7.15 percent in mid-May, based on their daily Zillow rate data.
UPDATED RESPONSE: 6.55
